In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sns

from anndata import AnnData
from scipy.stats import pearsonr, wilcoxon
from sklearn.metrics import mean_squared_error

import warnings 
warnings.filterwarnings('ignore') 

import stan

## Loading ST dataset

We use a Visium spatial transcriptomics dataset of the human lymph node, which is publicly available from [the 10x genomics website](https://support.10xgenomics.com/spatial-gene-expression/datasets/1.0.0/V1_Human_Lymph_Node). For first time use, the dataset will be downloaded to the directory `data/V1_Human_Lymph_Node` by default.

In [2]:
adata = sc.datasets.visium_sge(sample_id="V1_Human_Lymph_Node")
adata.var_names_make_unique()
adata

AnnData object with n_obs × n_vars = 4035 × 36601
    obs: 'in_tissue', 'array_row', 'array_col'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'spatial'
    obsm: 'spatial'

We perform some basic filtering of genes and spots.

In [3]:
sc.pp.filter_genes(adata, min_cells=5)
sc.pp.filter_cells(adata, min_counts=5000)

Write the processed data to file.

In [4]:
!mkdir results_lymphnode/
adata.write("results_lymphnode/lymphnode.h5ad")

mkdir: results_lymphnode/: File exists


## Loading the gene-TF prior matrix

We obtain a gene set resource comprising TF–target gene priors from [hTFtarget](https://guolab.wchscu.cn/hTFtarget/#!/), and retain only those TFs that were identified in the Human Transcription Factor database [(humantfs)](https://www.cell.com/cell/fulltext/S0092-8674(18)30106-5) to generate the TF–target gene prior matrix. For first time use, the prior information will be downloaded to the directory `data/gene_tf` by default.

For the ST dataset, the TF–target gene prior matrix (referred to as $D$) and the spot-level gene expression matrix (referred to as $Y$) undergo a sequential filtering process:
- In $Y$, genes expressed in less than 20% of the spots are removed. 
- Only mutual genes present in both matrices are retained. 
- In $D$, genes associated with less than 5 remaining TFs are removed, and TFs associated with fewer than 10 remaining genes are removed.

All paramters are tunable. The filtered TF–target gene prior matrix is stored in `adata.varm['gene_tf']`.

In [5]:
adata = stan.add_gene_tf_matrix(adata,
                                min_cells_proportion = 0.2,
                                min_tfs_per_gene= 5,
                                min_genes_per_tf= 10,
                                gene_tf_source="hTFtarget",
                                tf_list="humantfs",
                                source_dir="data/gene_tf/")

We check the sizes of the matrices after filtering.

In [6]:
D = adata.varm['gene_tf']
print('gene-TF matrix: {} x {}'.format(D.shape[0], D.shape[1]))
print('min tfs associated with each gene: {}'.format(D.T.abs().sum().min()))
print('min genes associated with each tf: {}'.format(D.abs().sum().min()))

gene-TF matrix: 8931 x 234
min tfs associated with each gene: 5
min genes associated with each tf: 10


In [7]:
Y = adata.to_df()
print('gene-cell matrix: {} x {}'.format(Y.shape[1], Y.shape[0]))
print('min cells associated with each gene: {}'.format((Y>0).sum().min()))
print('min genes associated with each cell: {}'.format((Y>0).T.sum().min()))

gene-cell matrix: 8931 x 3991
min cells associated with each gene: 799
min genes associated with each cell: 2132


## Computing the spatially dependent kernel

A Gaussian kernel matrix $K$,  stored in `adata.obsm['kernel']`, is computed based on the spatial and morphological information of the spot and a neighborhood of a square.
-  `windowsize`: length of the half side of the square (unit: pixel).
- `bandwidth`: band width using the computation of $K$.
- `im_feats_weight`: morphological info to spatial info ratio.
- `n`: the top $n$ singular values of $A$ in SVD.

All paramters are tunable. The spatial and morphological information is stored in `adata.obsm['spatial']` and `adata.obsm['pixel']` respectively, and are normalized to have zero mean and unit variance for each component.

In [8]:
stan.pixel_intensity(adata, windowsize=25)
stan.make_kernel(adata, n=250, im_feats_weight=0.05, bandwidth=0.2)

Time elapsed: 0.29 seconds
Time elapsed: 6.10 seconds


We normalize each spot by total counts over all genes, and then square root transform the total count to stabilize the variance.

In [9]:
sc.pp.normalize_total(adata)
adata.layers['scaled'] = np.sqrt(adata.to_df())

## GRN inference using Ridge regression (baseline)

10-fold cross validatoin

In [10]:
stan.assign_folds(adata, n_folds=10, random_seed=0)
ridge_model = stan.Ridge(adata, layer='scaled')

Grid search for parameters ranging from 1e-4 to 1e4
i.e. 1e-4, 1e-2, 1, 1e2, 1e4

In [11]:
ridge_model.fit(n_steps=5, stages=1, grid_search_params={'lam':[1e-4, 1e4]})

Time elapsed: 28.95 seconds


Evaluate the cross validation performance using Pearsonr correlation coefficient.

Note: computing correlation is time consuming.

In [12]:
cor, gene_cor = ridge_model.evaluate(fold=-1)
adata.obs['pred_cor_ridge'] = cor
adata.var['pred_cor_ridge'] = gene_cor

print(ridge_model.params)
print("Spot-wise correlation:" + str(round(np.nanmedian(cor), 4)))
print("Gene-wise correlation: " + str(round(np.nanmedian(gene_cor), 4)))

{'lam': 0.01}
Spot-wise correlation:0.2116
Gene-wise correlation: 0.07


Evaluate the validation performance using mean squared error.

In [13]:
Y = adata.varm['gene_tf'].dot(ridge_model.W_concat)
mean_squared_error(Y, adata.to_df('scaled').T)

1.0552204270068621

Store the inferred TF activity matrix in `adata.obsm['tfa_ridge']`.

In [14]:
adata.obsm['tfa_ridge'] = pd.DataFrame(
    ridge_model.W_concat.T, index=adata.obs_names, columns=adata.uns['tf_names'])

## GRN inference using STAN

The program finds the optimal parameters on a $5\times5$ grid, so it will take a few minutes.

In [15]:
stan_model = stan.Stan(adata, layer='scaled')
stan_model.fit(n_steps=5, stages=1,
              grid_search_params={'lam1':[1e-4, 1e4], 'lam2':[1e-4, 1e4]})

Time elapsed: 128.45 seconds


Evaluate the cross validation performance using Pearsonr correlation coefficient.

In [16]:
cor, gene_cor = stan_model.evaluate(fold=-1)
adata.obs['pred_cor_stan'] = cor
adata.var['pred_cor_stan'] = gene_cor

print(stan_model.params)
print("Spot-wise correlation:" + str(round(np.nanmedian(cor), 4)))
print("Gene-wise correlation: " + str(round(np.nanmedian(gene_cor), 4)))

{'lam2': 10000.0, 'lam1': 10000.0}
Spot-wise correlation:0.2281
Gene-wise correlation: 0.0919


Evaluate the validation performance using mean squared error.

In [17]:
Y = adata.varm['gene_tf'].dot(stan_model.W_concat)
mean_squared_error(Y, adata.to_df('scaled').T)

1.044083649293358

Store the inferred TF activity matrix in `adata.obsm['tfa_stan']`.

In [18]:
adata.obsm['tfa_stan'] = pd.DataFrame(
    stan_model.W_concat.T, index=adata.obs_names, columns=adata.uns['tf_names'])

## Writing the results to files

In [19]:
adata.write("results_lymphnode/lymphnode_stan.h5ad")